# [WORK IN PROGRESS]
# Temporal Attention Model for Neural Machine Translation
Unofficial implementation of paper: http://arxiv.org/abs/1608.02927

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


### Downloading (French, English) language pair.

In [5]:
from tensorflow.models.rnn.translate import data_utils
data_dir = "/data/translate" # You may need to change that or create a sympolic link
vocab_size = 20000
pathes = data_utils.prepare_wmt_data(data_dir, vocab_size, vocab_size)
en2_path, fr2_path, en2013_path, fr2013_path, en_vocab_path, fr_vocab_path = pathes

In [66]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_UNK = "_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

def read_vocab(vocab_path):
    vocab_list = []
    vocab_list.extend(_START_VOCAB)

    with open(vocab_path, 'br') as f:
        vocab_list.extend([s.decode("utf-8").strip() for s in f.readlines() if is_ascii(s)])

    words_to_ids = {w:i for (i, w) in enumerate(vocab_list)}
    ids_to_words = {i:w for (w, i) in words_to_ids.items()}
    return ids_to_words, words_to_ids

In [67]:
en_index, en_vocab = read_vocab(en_vocab_path)
fr_index, fr_vocab = read_vocab(fr_vocab_path)

def token_ids_to_sentence(ids, vocab_inv):
    return " ".join([vocab_inv.get(_id, _UNK) for _id in ids])

In [68]:
#FIXME: some non-ascii charachters
en_vocab_size = len(en_vocab) + 1
fr_vocab_size = len(fr_vocab) + 1

In [69]:
print(len(en_vocab))
print(len(en_index))

19905
19905


In [17]:
from data_utils import *

In [72]:
ids = sentence_to_token_ids("A is me strategy stratégie", en_vocab)
print(ids)
token_ids_to_sentence(ids, en_index)

[15239, 22, 1511, 614, 3]


'a is me strategy _UNK'

### Reading dataset

In [74]:
pathes

('/data/translate/giga-fren.release2.ids20000.en',
 '/data/translate/giga-fren.release2.ids20000.fr',
 '/data/translate/newstest2013.ids20000.en',
 '/data/translate/newstest2013.ids20000.fr',
 '/data/translate/vocab20000.en',
 '/data/translate/vocab20000.fr')

In [86]:
def read_data(path):
    with open(path, 'r') as f:
        return [[int(x) for x in line.split(" ")] for line in f.read().splitlines()]

In [87]:
en_set = read_data(en2013_path)
fr_set = read_data(fr2013_path)

In [91]:
en_maxlen = max(map(len, en_set))
fr_maxlen = max(map(len, fr_set))
print(en_maxlen)
print(fr_maxlen)

110
126


### Building the model

In [73]:
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector, Permute, Activation, recurrent, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [96]:
encoder = Sequential()
encoder.add(Embedding(en_vocab_size, 50, input_length=en_maxlen))
encoder.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [100]:
encoder.predict(np.array([x for x in en_set if len(x) == 110]))

array([[[-0.04156312,  0.01821918, -0.04753896, ..., -0.03579432,
          0.02079523, -0.04182146],
        [ 0.02747966,  0.01740393, -0.01015042, ...,  0.03865925,
          0.03215876, -0.00618281],
        [-0.02605469, -0.00345312, -0.0356955 , ..., -0.00068561,
          0.00706407,  0.00476055],
        ..., 
        [-0.02605469, -0.00345312, -0.0356955 , ..., -0.00068561,
          0.00706407,  0.00476055],
        [-0.04654215, -0.03888209,  0.03690505, ...,  0.02469091,
          0.04567369,  0.02441451],
        [ 0.03491415,  0.0054909 ,  0.00489568, ...,  0.00560037,
          0.03416372, -0.02973272]]], dtype=float32)